# Score de Risco 

### Criado por João Teles e Rodrigo Sales para discutir os resultados analíticos de como certo modelos de Machine Learning podem se comportar nos dados fonecidos pela empresa Praso.

#### Importação de frameworks para análise preliminar dos dados.

In [1]:

import pandas as pd


##### Primeiras impressões com tipo de dados, checar os valores nulos e aplicar estatística padrão

In [4]:
dados_clientes = pd.read_csv('amostra_praso.csv', sep=',')

# Exibe a quantidade de linhas e colunas do DataFrame
print(dados_clientes.shape)

# Exibe as primeiras linhas do DataFrame
print(dados_clientes.head())


(12458, 20)
   uf    municipio        segmento_cliente                natureza_juridica  \
0  PE       RECIFE  Restaurante Brasileiro  213-5 - Empresário (Individual)   
1  PE   CHA GRANDE          Pizza & Massas  213-5 - Empresário (Individual)   
2  PB  JOAO PESSOA     Doces & Confeitaria  213-5 - Empresário (Individual)   
3  CE     CASCAVEL      Lanches & Salgados  213-5 - Empresário (Individual)   
4  PE       RECIFE     Mercearia & Mercado  213-5 - Empresário (Individual)   

  fonte_cliente cnae_codigo  \
0       Fonte 5  56.11-2-01   
1       Fonte 2  56.11-2-01   
2       Fonte 2  10.93-7-01   
3       Fonte 2  56.20-1-04   
4       Fonte 2  56.11-2-03   

                                      cnae_descricao capital_social  \
0                           Restaurantes e similares  (8000, 10000]   
1                           Restaurantes e similares   (3000, 5000]   
2  Fabricação de produtos derivados do cacau e de...    (200, 1000]   
3  Fornecimento de alimentos preparados pr

In [5]:
# Informações gerais
dados_clientes.info()

# Estatísticas descritivas
dados_clientes.describe()

# Verificando valores nulos
dados_clientes.isnull().sum()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12458 entries, 0 to 12457
Data columns (total 20 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   uf                                12458 non-null  object 
 1   municipio                         12458 non-null  object 
 2   segmento_cliente                  12458 non-null  object 
 3   natureza_juridica                 12458 non-null  object 
 4   fonte_cliente                     12458 non-null  object 
 5   cnae_codigo                       12458 non-null  object 
 6   cnae_descricao                    12458 non-null  object 
 7   capital_social                    12458 non-null  object 
 8   idade_cnpj                        12458 non-null  object 
 9   serasa_contagem_negativacoes      12458 non-null  int64  
 10  serasa_contagem_protestos         12458 non-null  int64  
 11  serasa_idade_divida_mais_recente  2356 non-null   float64
 12  sera

uf                                      0
municipio                               0
segmento_cliente                        0
natureza_juridica                       0
fonte_cliente                           0
cnae_codigo                             0
cnae_descricao                          0
capital_social                          0
idade_cnpj                              0
serasa_contagem_negativacoes            0
serasa_contagem_protestos               0
serasa_idade_divida_mais_recente    10102
serasa_credores                     10483
serasa_socio_tem_negativacao            0
ifood_contagem_avaliacoes           11978
ifood_faixa_preco                   11604
google_maps_avaliacao               11593
google_maps_contagem_avaliacoes     11420
google_maps_tem_website             11420
inadimplente                            0
dtype: int64

##### Corrigir tipo de dados de capital_social e idade_cnpj para numérico.

In [17]:
# Garantir que a coluna é string antes de substituir caracteres
dados_clientes["capital_social"] = dados_clientes["capital_social"].astype(str).str.replace(r"\D", "", regex=True).astype(float)
dados_clientes["idade_cnpj"] = dados_clientes["idade_cnpj"].astype(str).str.replace(r"\D", "", regex=True).astype(float)

print(dados_clientes.dtypes)  # Verifique os tipos de dados das colunas



uf                                   object
municipio                            object
segmento_cliente                     object
natureza_juridica                    object
fonte_cliente                        object
cnae_codigo                          object
cnae_descricao                       object
capital_social                      float64
idade_cnpj                          float64
serasa_contagem_negativacoes          int64
serasa_contagem_protestos             int64
serasa_idade_divida_mais_recente    float64
serasa_credores                      object
serasa_socio_tem_negativacao          int64
ifood_contagem_avaliacoes            object
ifood_faixa_preco                    object
google_maps_avaliacao                object
google_maps_contagem_avaliacoes      object
google_maps_tem_website             float64
inadimplente                          int64
tem_divida                            int64
tem_ifood_avaliacoes                  int64
dtype: object


##### Como pode-se resolver o problema de dados nulos em diversas colunas?
##### Média/Mediana, caso os valores sejam distribuídos normalmente.
##### Zero, se fizer sentido como um dado ausente real.

In [18]:
dados_clientes["serasa_idade_divida_mais_recente"] = dados_clientes["serasa_idade_divida_mais_recente"].fillna(dados_clientes["serasa_idade_divida_mais_recente"].median())


##### Para Variáveis Categóricas
##### Podemos preencher valores nulos com "Desconhecido" ou "Não Informado".

In [19]:
dados_clientes["ifood_faixa_preco"] = dados_clientes["ifood_faixa_preco"].fillna("Não Informado")
dados_clientes["google_maps_avaliacao"] = dados_clientes["google_maps_avaliacao"].fillna(0)
dados_clientes["google_maps_tem_website"] = dados_clientes["google_maps_tem_website"].fillna(0)


##### Criar Indicadores de Nulos 
##### Criar uma feature binária para indicar ausência de informações.

In [11]:
dados_clientes["tem_divida"] = dados_clientes["serasa_idade_divida_mais_recente"].notnull().astype(int)
dados_clientes["tem_ifood_avaliacoes"] = dados_clientes["ifood_contagem_avaliacoes"].notnull().astype(int)


### Feature engineering

##### Agrupar o CNAE como sugerido, ...

In [20]:
dados_clientes["cnae_divisao"] = dados_clientes["cnae_codigo"].astype(str).str[:2]


#### Label Enconding para as variáveis com poucas categorias


#### One Hot Enconding para colunas com muitas categorias

In [21]:
dados_clientes = pd.get_dummies(dados_clientes, columns=["cnae_divisao"], drop_first=True)
